<a href="https://colab.research.google.com/github/trujoan/Cod_Interactivo/blob/main/calculadora_de_masa_molar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculadora de masa molar

In [1]:
!pip install periodictable rdkit-pypi pubchempy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 42.0 MB/s eta 0:00:00
  Created wheel for periodictable: filename=periodictable-1.7.1-py3-none-any.whl size=752513 sha256=86370f8c3a1f3db18b5309c07fa50a340d6aeeb27f4a8720818252e0a4cdd29a
  Stored in directory: /root/.cache/pip/wheels/55/2b/21/c35cc7b4e26fffa3ec4f32a74bd09ce6ce04ba7535dd0b92f2
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=9d851731216b24bae4a1a3ddb04f96b77d4efc097607bb1d0b9189f0e79d6ab9
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built periodictable pubchempy


In [3]:
import periodictable
import re
from rdkit import Chem
from rdkit.Chem import Draw
import pubchempy as pcp
from IPython.display import display

# Diccionario para nombres en español
compound_names_es = {
 'NaHCO3': 'Bicarbonato de sodio',
 'HCl': 'Ácido clorhídrico',
 'NaCl': 'Cloruro de sodio',
 'H2O': 'Agua',
 'CO2': 'Dióxido de carbono',
 }
# Función para traducir el nombre IUPAC a español (básico)
def translate_iupac_to_spanish(iupac_name):
 translations = {
 'sodium': 'sodio',
 'hydrogen': 'hidrógeno',
 'carbonate': 'carbonato',
 'chloride': 'cloruro',
 'water': 'agua',
 'dioxide': 'dióxido',
 }
words = iupac_name.split(';')
 translated = [translations.get(word.strip(), word.strip()) for word in words]
 return ' y '.join(translated)

 def parse_formula(formula): #analiza una formula quimica y devuelve un
diccionario con la cantidad de átomos de cada elemento. Usa expresiones regulares
para identificar elementos y cantidades en la fórmula.
pattern = r'([A-Z][a-z]?)(\d*)'
 matches = re.findall(pattern, formula)
 element_counts = {}
 for (element, count) in matches:
 if count == '':
 count = 1
 else:
 count = int(count)
 element_counts[element] = element_counts.get(element, 0) + count
 return element_counts
def calculate_molar_mass(formula):
 element_counts = parse_formula(formula)
 molar_mass = 0.0
 for element, count in element_counts.items():
 try:
 el = getattr(periodictable, element)
 molar_mass += el.mass * count
 except AttributeError:
 print(f"Elemento {element} no encontrado.")
 return molar_mass
 def draw_molecule(formula):
 try:
 compound = pcp.get_compounds(formula, 'formula')[0]
 smiles = compound.isomeric_smiles
 mol = Chem.MolFromSmiles(smiles)
 img = Draw.MolToImage(mol)
 return img, compound
 except IndexError:
 print(f"No se encontraron compuestos para la fórmula: {formula}")
 return None, None
except Exception as e:
 print(f"Error al dibujar la molécula: {e}")
 return None, None
 def parse_reaction(reaction_str):
 compounds = re.split(r'\s*\+\s*', reaction_str)
 return compounds
def display_reaction_structures(compounds, moles):
 total_molar_mass = 0.0
 for compound in compounds:
 print(f"\nFórmula: {compound}")
 molar_mass = calculate_molar_mass(compound)
 print(f"Masa molar de {compound}: {molar_mass:.2f} g/mol")
 total_molar_mass += molar_mass
 # Obtener el nombre en español
 name_es = compound_names_es.get(compound, 'Nombre desconocido')
 print(f"Nombre en español: {name_es}")
 # Obtener el nombre IUPAC y traducirlo
 image, info_compuesto = draw_molecule(compound)
 if image and info_compuesto:
 display(image)
 iupac_name = info_compuesto.iupac_name
 translated_name = translate_iupac_to_spanish(iupac_name)
 print(f"Nombre IUPAC en español: {translated_name}")
 print(f"Sinónimos: {', '.join(info_compuesto.synonyms)}")
 total_mass = total_molar_mass * moles
 print(f"\nMasa molar total de los compuestos: {total_molar_mass:.2f} g/mol")
 print(f"Masa total para {moles} moles: {total_mass:.2f} g")
 def predict_reaction(compounds):
 reactions = {
 frozenset(['NaHCO3', 'HCl']): ['NaCl', 'H2O', 'CO2'], # Reacción de
bicarbonato de sodio y HCl
 }
compound_set = frozenset(compounds)
for reactants, products in reactions.items():
 if reactants.issubset(compound_set):
 return products
 return None
def main():
 print("Bienvenido a la calculadora de masa molar y estructura química.")
 print("Este programa facilita el estudio de la química al proporcionar
información sobre compuestos inorgánicos.")
 reaction_str = input("Ingrese la fórmula química del compuesto o reacción
deseada (solo compuestos inorgánicos): ")
 compounds = parse_reaction(reaction_str)
 moles = float(input("Ingrese la cantidad de moles para evaluar la masa molar:
"))

print("\nMostrando estructuras de los reactivos:")
 display_reaction_structures(compounds, moles)
 products = predict_reaction(compounds)
 if products:
 print("\nPrediciendo productos de la reacción:")
 display_reaction_structures(products, moles)
 print(f"\nReacción: {' + '.join(compounds)} → {' + '.join(products)}")
 else:
 print("No se encontraron productos para esta combinación de reactivos.")
if __name__ == "__main__":
 main()


IndentationError: unexpected indent (<ipython-input-3-d1e560dc3a09>, line 27)